In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
# mpl.use("pdf")
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mpl.rcParams["font.serif"] = "CMU serif"
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "CMU serif"
mpl.rcParams["mathtext.it"] = "CMU serif:italic"
mpl.rcParams["mathtext.bf"] = "CMU serif:bold"
mpl.rcParams["font.family"] = "serif"

# import pylandau
from scipy.optimize import curve_fit
import scipy
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf
# import ROOT

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
# import PlotLib.Stats as Stats
import PlotLib.Hist1D as Hist1D
import PlotLib.Hist2D as Hist2D
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

import time

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

import ROOT

def get_mask_seedPix(data):
    return data[:,:,M["Charge"]] == arr([np.max(data[:,:,M["Charge"]], axis=1) for i in range(4)]).T

In [ ]:
nEvents = 4*50000
# nEvents = 4*200
runIDs = arr([184,190,192])
runIDs = arr([190])
filepaths = {
    184:f"/home/jona/DESY/analysis_TB/output/csv/184_thr200.csv",
    190:f"/home/jona/DESY/analysis_TB/output/csv/190_thr200.csv",
    192:f"/home/jona/DESY/analysis_TB/output/csv/190_thr520.csv",
}

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, nEvents*4, filepaths[runID])
    # runs[runID]["data"] has the following structure:
    # [ event , pixel, entry ] where entry is the index given by the map M
    
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],nEvents)
    runs[runID]["threshold"] = 200 if runID in [184,190] else 520
    runs[runID]["MTP"] = 5
    
M = runs[runIDs[0]]["M"]

def findMax(func, par, x0=10):
    def f(x, par):
        return -1 * func(x, par)
    f2 = np.vectorize(f, excluded=[1])
    return scipy.optimize.fmin(f2, x0, args=(par,), disp=False)[0]

print(M)

In [ ]:
# Charge vs ToT
runID = 190

pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])

binN = arr([7,5])*8
binRange = arr([[0,600], [0,250]])
zRange = arr([0,200])

FileName = "Charge_vs_ToT"
fig, axs = Plot.create_fig(2,2, sharex=True, sharey=True, flatten=False) 

for i_row in range(2):
    for i_col in range(2):
        i_pix = i_row*2+i_col
        i_pix = pixMap[i_pix]
        mask = runs[runID]["data"][:,:,M["Charge"]] > runs[runID]["threshold"]
        mask = np.logical_and(mask, get_mask_seedPix(runs[runID]["data"]))
        mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixCol"]] == i_col)
        mask = np.logical_and(mask, runs[runID]["data"][:,:,M["PixRow"]] == i_row)
        
        entriesX = runs[runID]["data"][:,:,M["ToT"]].flatten()[mask.flatten()]
        entriesY = runs[runID]["data"][:,:,M["Amplitude"]].flatten()[mask.flatten()]*1000
        
        hist = Hist2D.Hist_2D(binN, binRange)
        hist.fill(entriesX, entriesY)
        
        # hist.draw(axs[i_row][i_col], aspect="auto", cmap="viridis", vmin=zRange[0], vmax=zRange[1], ax_cbar=None)
        hist.draw(axs[i_row][i_col], aspect="auto", cmap="viridis", ax_cbar=None)
        
        Fe55_charge = 1613
        
        

fig.subplots_adjust(wspace=0.05, hspace=0.05)
        
    
Plot.finalize(runs[runID], fig, axs.flatten(), 
    title=r"Amplitude vs. ToT",
    xlabel=r"Seed Pixel Time over Threshold / ns",
    ylabel= r"Seed Pixel Amplitude / mV",
    param_dict={"campaign":True, "sample":True, "ER1Param":True, "recoParam":False, "fontsize":8, "thr":runs[runID]["threshold"]},
    grid=False, legend_loc="upper right")

Plot.savefig(fig, FileName)